### Imports

In [2]:
# APIs
import sys 
sys.path.append('..\..\src')
from paths import PARENT_DIR

from dotenv import load_dotenv


load_dotenv(PARENT_DIR / '.env')

True

In [3]:
import os
import pinecone

# Initialize Pinecone
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment="gcp-starter")

c:\Users\614019146\Documents\langchain_basics\rag_env\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Creating Prompts

1) Document prompt : This is a template, which instructs how results from pinecone should be formatted.
2) Question prompt : This is the template which ultimately is given as input to the llm. It incorporates the document prompt. 



In [27]:
from langchain.prompts import PromptTemplate

DOCUMENT_PROMPT  = """{page_content}"""


QUESTION_PROMPT = """ You have been asked a question about tv shows, typically to recommend some shows. Below is some information 
extracted from a tv-show database, and it will help you answer the question.

Context:
{context}

The question : {question}

answer: 
"""

document_prompt = PromptTemplate.from_template(DOCUMENT_PROMPT)
question_prompt = PromptTemplate.from_template(QUESTION_PROMPT)

### Using langchain to perform RAG

In [13]:
# imports for making pinecone retriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

# imports to build chain 
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from langchain.chat_models import ChatOpenAI


In [28]:
# Initialise the embeddings model
embeddings = OpenAIEmbeddings()

# Create the DocSearch object using the Pinecone index
docsearch = Pinecone.from_existing_index("tmdb-tv-shows", embeddings.embed_query)

# choose llm to answer question 
model = "gpt-3.5-turbo"

# Create chatbot, takes in llm, chain format, and retriever. "stuff" is a simple chain 
qa_bot = RetrievalQA.from_chain_type(
    chain_type="stuff",
    llm=ChatOpenAI(model_name=model, temperature=0),
    chain_type_kwargs={
        "document_prompt": document_prompt,
        "prompt": question_prompt,
    },
    retriever=docsearch.as_retriever(),
)


c:\Users\614019146\Documents\langchain_basics\rag_env\lib\site-packages\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [50]:
question = "Can you recommend a comedy show?"

# Ask the question 
answer = qa_bot(question)

In [51]:
from IPython.display import display, Markdown

display(Markdown(answer['result']))

Based on the information provided, I would recommend checking out "Comedy Showcase" on Channel 4. It features one-off comedy specials with up-and-coming talent, similar to the format of the earlier Comedy Playhouse. It is a returning series, so you can enjoy a variety of comedic performances from Britain's fledgling comedy talent.

#### Updating the question prompt

Updating the prompt, so it includes an example q and a.

In [54]:
QUESTION_PROMPT_2 = """ 
You have been asked a question about tv shows, and below is some information 
extracted from a tv-show database which should help you answer. 

===============
Here is an example question and answer: 

QUESTION : 
Can you recommend a sci-fi tv show? 
================
Name: Dark/Web                               
Genres: Sci-Fi & Fantasy                               
Overview: A horror anthology series that explores the dangers of a totally connected world.                                
Status: Ended                                
Network: Prime Video 
=========
Name: Star Trek: Prodigy                               
Genres: Animation, Sci-Fi & Fantasy, Kids                               
Overview: A motley crew of young rebellious aliens commandeer an old Starfleet ship and must figure out how to work together while navigating a greater galaxy, in search for a better future. These six young outcasts know nothing about the ship they have commandeered, but over the course of their adventures together, they will each be introduced to Starfleet and the ideals it represents.                                
Status: Canceled                                
Network: Paramount+ 
=========
Name: Dungeons & Dragons                               
Genres: Animation, Sci-Fi & Fantasy, Comedy, Action & Adventure, Kids                               
Overview: Six friends are transported into the Dungeons & Dragons realm and must try to find a way home with the help of their guide 'Dungeon Master'.                                
Status: Canceled                                
Network: CBS 
=========
ANSWER : Dark/Web is a sci-fi and fantasy show which illustrates the dangers of a digitially connected world. You can watch it on Prime. 

=======

Context:
{context}

The question : {question}

answer: 
"""

question_prompt_2 = PromptTemplate.from_template(QUESTION_PROMPT_2)

In [55]:
# Create chatbot, takes in llm, chain format, and retriever. "stuff" is a simple chain 
qa_bot_2 = RetrievalQA.from_chain_type(
    chain_type="stuff",
    llm=ChatOpenAI(model_name=model, temperature=0),
    chain_type_kwargs={
        "document_prompt": document_prompt,
        "prompt": question_prompt_2,
    },
    retriever=docsearch.as_retriever(),
)

In [56]:
display(Markdown(qa_bot_2(question)['result']))

If you're looking for a good comedy show, you can check out "The Comedy Lineup" on Netflix. It features up-and-coming comedians performing 15-minute sets in a stand-up comedy showcase series.